# VGGNet
Training example using MNIST dataset

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
data_builder = tfds.builder("cifar100")
data_builder.download_and_prepare()

In [ ]:
train_dataset = data_builder.as_dataset(split=tfds.Split.TRAIN)
val_dataset = data_builder.as_dataset(split=tfds.Split.TEST)

num_classes = data_builder.info.features['label'].num_classes

num_train = data_builder.info.splits['train'].num_examples
num_val = data_builder.info.splits['test'].num_examples

print('# for train : %d'%(num_train))
print('# for valid : %d'%(num_val))

In [ ]:
input_shape = [224, 224, 3]

batch_size = 16
num_epochs = 10

## Create model

In [ ]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

import functools

In [ ]:
Conv3_64 = functools.partial(Conv2D,
                            filters=64,
                            kernel_size=(3, 3),
                            padding='same',
                            activation='relu')

Conv3_128 = functools.partial(Conv2D,
                             filters=128,
                             kernel_size=(3, 3),
                             padding='same',
                             activation='relu')

Conv3_256 = functools.partial(Conv2D,
                             filters=256,
                             kernel_size=(3, 3),
                             padding='same',
                             activation='relu')

Conv3_512 = functools.partial(Conv2D,
                             filters=512,
                             kernel_size=(3, 3),
                             padding='same',
                             activation='relu')
Dense_4096 = functools.partial(Dense,
                          units=4096, 
                          kernel_regularizer=tf.keras.regularizers.L2(0.0005),
                          activation='relu')

In [ ]:
def VGGNet(input_shape, num_classes, model_type: 16 or 19 =16):    
    input = Input(shape=input_shape, name='Input')
    
    x = Conv3_64(name='block1_conv1')(input)
    x = Conv3_64(name='block1_conv2')(x)
    x = MaxPooling2D(pool_size=2, padding='same', name='block1_pool')(x)
    
    x = Conv3_128(name='block2_conv1')(x)
    x = Conv3_128(name='block2_conv2')(x)
    x = MaxPooling2D(pool_size=2, padding='same', name='block2_pool')(x)
    
    x = Conv3_256(name='block3_conv1')(x)
    x = Conv3_256(name='block3_conv2')(x)
    x = Conv3_256(name='block3_conv3')(x)
    if model_type == 19:
        x = Conv3_256(name='block3_conv4')(x)
    x = MaxPooling2D(pool_size=2, padding='same', name='block3_pool')(x)
    
    x = Conv3_512(name='block4_conv1')(x)
    x = Conv3_512(name='block4_conv2')(x)
    x = Conv3_512(name='block4_conv3')(x)
    if model_type == 19:
        x = Conv3_512(name='block4_conv4')(x)
    x = MaxPooling2D(pool_size=2, padding='same', name='block4_pool')(x)
    
    x = Conv3_512(name='block5_conv1')(x)
    x = Conv3_512(name='block5_conv2')(x)
    x = Conv3_512(name='block5_conv3')(x)
    if model_type == 19:
        x = Conv3_512(name='block5_conv4')(x)
    x = MaxPooling2D(pool_size=2, padding='same', name='block5_pool')(x)
    
    x = Flatten(name='flatten')(x)
    x = Dense_4096(name='fc1')(x)
    x = Dropout(0.5)(x)
    x = Dense_4096(name='fc2')(x)  
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax', name='ouput')(x)

    model = Model(inputs=input, outputs=output)
    if model_type == 19:
        model._name = 'VGG19'
    else:
        model._name = 'VGG16'
    
    return model

In [ ]:
def VGG16(input_shape, num_classes):
    return VGGNet(input_shape=input_shape,
                 num_classes=num_classes,
                 model_type=16)

def VGG19(input_shape, num_classes):
    return VGGNet(input_shape=input_shape,
                 num_classes=num_classes,
                 model_type=19)

In [ ]:
batch_input_shape = tf.TensorShape((None, *input_shape))

In [ ]:
model = VGG16(input_shape, num_classes)
model.build(input_shape=batch_input_shape)
model.summary()

In [ ]:
model = VGG19(input_shape, num_classes)
model.build(input_shape=batch_input_shape)
model.summary()

In [ ]:
input_shape = [32, 32, 3]

batch_size = 16
num_epochs = 40

In [ ]:
def MiniVGGNet(input_shape, num_classes):
    input = Input(shape=input_shape, name='Input')
    
    x = Conv3_64(name='block1_conv1')(input)
    x = Conv3_64(name='block1_conv2')(x)
    x = MaxPooling2D(pool_size=2, padding='same', name='block1_pool')(x)
    
    x = Conv3_128(name='block2_conv1')(x)
    x = Conv3_128(name='block2_conv2')(x)
    x = MaxPooling2D(pool_size=2, padding='same', name='block2_pool')(x)
    
    x = Conv3_256(name='block3_conv1')(x)
    x = Conv3_256(name='block3_conv2')(x)
    x = Conv3_256(name='block3_conv3')(x)
    x = MaxPooling2D(pool_size=2, padding='same', name='block3_pool')(x)
    
    x = Conv3_512(name='block4_conv1')(x)
    x = Conv3_512(name='block4_conv2')(x)
    x = Conv3_512(name='block4_conv3')(x)
    x = MaxPooling2D(pool_size=2, padding='same', name='block4_pool')(x)
    
    x = Conv3_512(name='block5_conv1')(x)
    x = Conv3_512(name='block5_conv2')(x)
    x = Conv3_512(name='block5_conv3')(x)
    x = MaxPooling2D(pool_size=2, padding='same', name='block5_pool')(x)
    
    x = Flatten(name='flatten')(x)
    x = Dense(512, activation='relu',
             kernel_regularizer=tf.keras.regularizers.L2(0.0005),
             name='fc1')(x)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax', name='ouput')(x)

    model = Model(inputs=input, outputs=output)
    model._name = 'VGG_mini'
    
    return model

In [ ]:
model = MiniVGGNet(input_shape, num_classes)
model.build(input_shape=batch_input_shape)
model.summary()

## Prepare training dataset

In [ ]:
def prepare_data_fn(features, input_shape, augment=False):
    
    input_shape = tf.convert_to_tensor(input_shape)
    
    image = features['image']
    label = features['label']
    image = tf.image.convert_image_dtype(image, tf.float32)
    
    if augment:
        image = tf.image.random_flip_left_right(image)
        
        image = tf.image.random_brightness(image, max_delta=0.1)
        image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
        image = tf.clip_by_value(image, 0.0, 1.0)
        
        random_scale_factor = tf.random.uniform([1], minval=1., maxval=1.4, dtype=tf.float32)
        scaled_height = tf.cast(tf.cast(input_shape[0], tf.float32) * random_scale_factor, tf.int32)
        scaled_width = tf.cast(tf.cast(input_shape[1], tf.float32) * random_scale_factor, tf.int32)
        scaled_shape = tf.squeeze(tf.stack([scaled_height, scaled_width]))
        image = tf.image.resize(image, scaled_shape)
        image = tf.image.random_crop(image, input_shape)
    else:
        image = tf.image.resize(image, input_shape[:2])
    return image, label

In [ ]:
import functools

prepare_data_fn_for_train = functools.partial(prepare_data_fn,
                                             input_shape=input_shape,
                                             augment=True)
prepare_data_fn_for_val = functools.partial(prepare_data_fn,
                                           input_shape=input_shape,
                                           augment=False)

train_dataset = train_dataset.repeat(num_epochs) \
                    .shuffle(10000) \
                    .map(prepare_data_fn_for_train, num_parallel_calls=4) \
                    .batch(batch_size) \
                    .prefetch(1)

val_dataset = val_dataset.repeat() \
                .map(prepare_data_fn_for_val, num_parallel_calls=4) \
                .batch(batch_size) \
                .prefetch(1)

## Tensorboard

In [ ]:
model_dir = './models/vggnet'

In [ ]:
cifar100 = tf.keras.datasets.cifar100
(_, _), (visual_images, visual_labels) = \
    cifar100.load_data()

In [ ]:
str_labels = ['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman', 'worm']

In [ ]:
import io
import random

import numpy as np
import matplotlib.pyplot as plt

file_writer = tf.summary.create_file_writer(model_dir)

def plot_to_image(figure):
      """Converts the matplotlib plot specified by 'figure' to a PNG image and
      returns it. The supplied figure is closed and inaccessible after this call."""
      # Save the plot to a PNG in memory.
      buf = io.BytesIO()
      plt.savefig(buf, format='png')
      # Closing the figure prevents it from being displayed directly inside
      # the notebook.
      plt.close(figure)
      buf.seek(0)
      # Convert PNG buffer to TF image
      image = tf.image.decode_png(buf.getvalue(), channels=4)
      # Add the batch dimension
      image = tf.expand_dims(image, 0)
      return image

def image_grid():
    test_images = tf.keras.applications.vgg16.preprocess_input(visual_images)
    pred = model.predict(test_images)
    pred = np.argmax(pred, axis=1)
    figure = plt.figure(figsize=(10,10))
    
    random_index = random.sample(range(len(visual_images)), 25)
    for i in range(25):
        # Start next subplot.
        plt.subplot(5, 5, i + 1, title=str_labels[pred[random_index[i]]])
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(visual_images[random_index[i]], cmap=plt.cm.binary)

    return figure

def log_images(epoch, logs):
    figure = image_grid()
    with file_writer.as_default():
        tf.summary.image("25 test data examples", plot_to_image(figure), step=0)

## Training

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01,momentum=0.9,nesterov=True)
accuracy_metric = tf.metrics.SparseCategoricalAccuracy(name='acc')
top5_accuracy_metric = tf.metrics.SparseTopKCategoricalAccuracy(k=5, name='top5_acc')

In [ ]:
model.compile(optimizer=optimizer,
             loss='sparse_categorical_crossentropy',
             metrics=[accuracy_metric, top5_accuracy_metric])

In [ ]:
import os

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss',
                                    restore_best_weights=True),
    
    tf.keras.callbacks.TensorBoard(log_dir=model_dir, 
                                   histogram_freq=0, 
                                   write_graph=True,
                                   write_images=True,
                                   update_freq=100),
    
    tf.keras.callbacks.ModelCheckpoint(
        os.path.join(model_dir, 'weights-epoch{epoch:02d}.h5')),
    
    tf.keras.callbacks.LambdaCallback(on_epoch_end=log_images)
]

In [ ]:
import math

train_steps_per_epoch = math.ceil(num_train / batch_size)
val_steps_per_epoch = math.ceil(num_val / batch_size)

history = model.fit(train_dataset,
                        epochs=num_epochs, steps_per_epoch=train_steps_per_epoch,
                        validation_data=(val_dataset),
                        validation_steps=val_steps_per_epoch,
                        verbose=1, callbacks=callbacks)